In [1]:
#@title  **Alternative Sankey Diagram with Louvain Communities**



# --- [الخطوة 0: التثبيت والاستيراد] ---
print("--- [Step 0: Installing Libraries] ---")
!pip uninstall -y community > /dev/null 2>&1
!pip install -qq python-louvain plotly seaborn networkx pandas
!git clone https://github.com/NoorBayan/Amthal.git
%cd Amthal
from utility import *
from IPython.display import display, HTML, clear_output
print("Libraries installed successfully.\n")


# --- [الخطوة 1: الاستيراد] ---
print("--- [Step 1: Importing Libraries] ---")
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import networkx as nx
import community.community_louvain as community_louvain
import seaborn as sns
from itertools import combinations
print("Libraries imported successfully.\n")




# --- [الخطوة 1: تحميل وتجهيز البيانات] ---
print("--- [Step 1: Loading Data] ---")
# ⚠️⚠️⚠️  الرجاء التأكد من أن هذا المسار صحيح في بيئتك
INSTANCES_FILE_PATH = '/content/Amthal/data/processed/instances.csv'
try:
    df = pd.read_csv(INSTANCES_FILE_PATH, sep='\t', encoding='utf-16')
    df['Dominant_Concept']=[Dominant_Concept_dic[i]['en'] for i in df.Dominant_Concept]
    df['Valence']=[Valence_dic[i]['ar'] for i in df.Valence]

    print(f"Data loaded successfully. Found {len(df)} instances.\n")
except FileNotFoundError:
    print(f"⚠️ ERROR: File not found at '{INSTANCES_FILE_PATH}'. Please check the path and try again.")
    exit()


# --- [الخطوة 3: بناء الشبكة واكتشاف المجتمعات (للتوافق)] ---
# (هذه الخطوة ضرورية فقط للحصول على نفس كائن 'partition' المستخدم في 4a)
print("--- [Step 3: Building Network and Detecting Communities] ---")
grouped = df.groupby(['Sura_No', 'Aya_No'])['Dominant_Concept'].apply(list)
edges = [tuple(sorted(combo)) for concepts in grouped if len(concepts) > 1 for combo in combinations(set(concepts), 2)]
edge_weights = pd.DataFrame(edges, columns=['source', 'target']).value_counts().reset_index(name='weight')

G = nx.from_pandas_edgelist(edge_weights, 'source', 'target', 'weight')
partition = community_louvain.best_partition(G, weight='weight', resolution=1.0, random_state=42)
partition_df = pd.DataFrame(list(partition.items()), columns=['Dominant_Concept', 'Community_ID'])
print("Network partitioned successfully.\n")


# --- [الخطوة 4: تجميع البيانات وتعيين التسميات الجديدة] ---
print("--- [Step 4: Aggregating Data and Assigning New Labels] ---")
instance_counts = df['Dominant_Concept'].value_counts().reset_index()
instance_counts.columns = ['Dominant_Concept', 'Instance_Count']

sankey_data = pd.merge(instance_counts, partition_df, on='Dominant_Concept')

# ✅✅✅  استخدام التسميات الجديدة والصحيحة للمجتمعات الثلاث
community_labels_new = {
    0: 'Axis of Power & Socio-Moral Order',
    1: 'Dynamics of Spiritual Existence',
    2: 'Cosmological & Symbolic Forces'
}
sankey_data['Community_Label'] = sankey_data['Community_ID'].map(community_labels_new)
print("New community labels assigned.\n")


# --- [الخطوة 5: تجهيز البيانات لمخطط Sankey] ---
print("--- [Step 5: Preparing Data for Plotly Sankey] ---")
# فرز المفاهيم حسب عدد المثيلات لتنظيم المخطط
sankey_data = sankey_data.sort_values(by='Instance_Count', ascending=False)

all_labels = pd.concat([
    sankey_data['Dominant_Concept'],
    sankey_data['Community_Label']
]).unique().tolist()

source_indices = [all_labels.index(concept) for concept in sankey_data['Dominant_Concept']]
target_indices = [all_labels.index(label) for label in sankey_data['Community_Label']]
values = sankey_data['Instance_Count']
print("Sankey data prepared.\n")
# --- [الخطوة 6: إنشاء المخطط بتنسيق أكاديمي لكل مجتمع لون مميز] ---
print("--- [Step 6: Generating Academic-Style Sankey Diagram] ---")

# 🎨 استخدام لوحة ألوان هادئة (Muted palette) من seaborn
num_communities = len(community_labels_new)
community_colors_palette = sns.color_palette("muted", num_communities)

# تحويل الألوان إلى صيغة RGBA بدرجة شفافية مناسبة للنشر
community_colors_rgba = [
    f'rgba({int(r*255)}, {int(g*255)}, {int(b*255)}, 0.85)' for r, g, b in community_colors_palette
]

# تعيين لون فريد لكل مجتمع
community_color_map = {
    'Axis of Power & Socio-Moral Order': community_colors_rgba[0],   # مثلاً رمادي مزرق
    'Dynamics of Spiritual Existence': community_colors_rgba[1],      # أخضر فاتح هادئ
    'Cosmological & Symbolic Forces': community_colors_rgba[2]        # بنفسجي باهت
}

# إنشاء ألوان العقد (المجتمعات + المفاهيم)
node_colors = []
for label in all_labels:
    if label in community_color_map:
        node_colors.append(community_color_map[label])
    else:
        node_colors.append('rgba(210, 210, 210, 0.9)')  # رمادي فاتح للمفاهيم

# إنشاء ألوان الروابط بنفس لون المجتمع ولكن بدرجة شفافية أكبر
link_colors = [
    community_color_map[sankey_data.iloc[i]['Community_Label']].replace('0.85', '0.35')
    for i in range(len(sankey_data))
]

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=14,
        thickness=20,
        line=dict(color="rgba(80,80,80,0.6)", width=0.6),
        label=all_labels,
        color=node_colors
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values,
        color=link_colors
    )
)])

# --- [إعدادات الشكل النهائية للنشر] ---
fig.update_layout(
    title_text="Distribution of Conceptual Instances into Thematic Communities",
    font=dict(size=12, family="Times New Roman", color="rgba(40,40,40,1)"),
    title_font=dict(size=15, family="Times New Roman", color="rgba(20,20,20,1)"),
    title_x=0.5,
    width=900,    # مناسب لعمود واحد في الورقة
    #width=420,    # مناسب لعمود واحد في الورقة
    height=600,
    margin=dict(l=10, r=10, t=50, b=10),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

clear_output()

fig.show()
print("✅ Academic-style Sankey diagram generated successfully.")


✅ Academic-style Sankey diagram generated successfully.
